In [1]:
import sys
import os

# Add the parent directory to sys.path if models.py is in the parent folder
sys.path.append(os.path.abspath('..'))

from models import GatheringModel, read_input, compute_multiphase_pressure_drop
import pandas as pd
from gamspy import Options, Ord

In [2]:
m = GatheringModel("Prototipo", data= read_input("toy_problem.xlsx"))

--- Job _FoMwnssAQZ2IJ5dTCCHDkw.gms Start 09/10/25 10:57:21 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\tmpiv_pwxvb\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\
    LimRow 0
    LimCol 0
    TraceOpt 3
    SolPrint 0


In [ ]:
m.solution_algorithm(scenario_name="prueba", max_iterations=10, solver_opts={"solver": "gurobi", "gap": 0.0001, "max_time": 1000.0})

--- Iteration 1 ---
--- _FoMwnssAQZ2IJ5dTCCHDkw.gms(1039) 10 Mb
--- Job _FoMwnssAQZ2IJ5dTCCHDkw.gms Start 09/10/25 10:57:52 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIQCP gurobi
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\tmpiv_pwxvb\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\

[CONVERTER - INFO] GAMS (.gms) file has been generated under prueba/iteration_1\Multiphase_network_design.gms


--- _FoMwnssAQZ2IJ5dTCCHDkw.gms(1109) 157 Mb
--- Job _FoMwnssAQZ2IJ5dTCCHDkw.gms Start 09/10/25 11:00:05 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\tmpiv_pwxvb\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\
    LimRow 

[CONVERTER - INFO] GAMS (.gms) file has been generated under prueba/iteration_2\Multiphase_network_design.gms


--- _FoMwnssAQZ2IJ5dTCCHDkw.gms(1233) 191 Mb
--- Job _FoMwnssAQZ2IJ5dTCCHDkw.gms Start 09/10/25 11:03:35 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\tmpiv_pwxvb\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\
    LimRow 

[CONVERTER - INFO] GAMS (.gms) file has been generated under prueba/iteration_3\Multiphase_network_design.gms


--- _FoMwnssAQZ2IJ5dTCCHDkw.gms(1357) 191 Mb
--- Job _FoMwnssAQZ2IJ5dTCCHDkw.gms Start 09/10/25 11:04:40 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\tmpiv_pwxvb\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\_FoMwnssAQZ2IJ5dTCCHDkw.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmpxynoakux\
    LimRow 

(False, 3)

In [4]:
ylp_inf = ((0.358819**(1/4.12))+1)**4.12

In [7]:
m.m["loc_x"].records

In [5]:
ylp_inf

10.751957060962651

In [4]:
sel_connections = m.m["sel_pipes"].records

In [5]:
sel_connections

,n,nn,d,element_text,last_selected
0,j2,pf2,d1,,True
1,j3,pf2,d1,,True
2,j5,pf2,d1,,True
3,j7,pf2,d1,,True
4,j8,pf2,d1,,True
5,j10,pf2,d1,,True


In [7]:
last_connections = m.obtain_var_df("x_bar")
last_connections = last_connections[last_connections["level"] > 0.5]

In [4]:
dist_df = m.obtain_var_df("dist", value_col="value")

In [ ]:
dist_df[]

,n,nn,value
0,i1,j1,0.000000
1,i2,j1,0.250000
2,i3,j1,0.500000
3,i4,j1,0.750000
4,i5,j1,2.000000
...,...,...,...
175,j8,pf2,2.490607
176,j9,pf2,1.305038
177,j10,pf2,1.256234
178,j11,pf2,1.256234


In [8]:
last_connections

,n,nn,d,t,level
162,i1,j2,d1,t15,1.0
5925,i2,j2,d1,t18,1.0
11910,i3,j3,d3,t21,1.0
17266,i4,j3,d1,t23,1.0
23706,i5,j5,d3,t1,1.0
29094,i6,j5,d1,t35,1.0
35520,i7,j7,d2,t36,1.0
41429,i8,j8,d2,t37,1.0
47412,i9,j10,d1,t1,1.0
53295,i10,j10,d4,t13,1.0


In [9]:
last_connections_unique = last_connections[["n","nn","d"]].drop_duplicates()

In [10]:
check = sel_connections.merge(last_connections_unique, on=["n","nn","d"], how="left", indicator=True)
sel_connections['exists_in_df2'] = check['_merge'] == 'both'

In [11]:
check

,n,nn,d,element_text,_merge
0,j2,pf2,d1,,both
1,j3,pf2,d1,,both
2,j5,pf2,d1,,both
3,j7,pf2,d1,,both
4,j8,pf2,d1,,both
5,j10,pf2,d1,,both


In [12]:
sel_connections

,n,nn,d,element_text,exists_in_df2
0,j2,pf2,d1,,True
1,j3,pf2,d1,,True
2,j5,pf2,d1,,True
3,j7,pf2,d1,,True
4,j8,pf2,d1,,True
5,j10,pf2,d1,,True


In [18]:
ixlm_ub = m.m["ixlm_ub"]
pw_list = m.m["pw"].records["uni"]
ixlm_values = m.obtain_record_values("ixlm_ub", [(pw, "j1", "pf1", "d1","t1") for pw in pw_list], var_bool=False)

In [60]:
ixlm_df = m.obtain_var_df("ixlm_ub", value_col="value")

In [62]:
ixlm_values

{('pw1', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw2', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw3', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw4', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw5', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw6', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw7', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw8', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw9', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw10', 'j1', 'pf1', 'd1', 't1'): None}

In [19]:
ixlm_values

{('pw1', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw2', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw3', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw4', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw5', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw6', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw7', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw8', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw9', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw10', 'j1', 'pf1', 'd1', 't1'): None}

In [54]:
last_connections

,n,nn,d,t,level
88,i1,j1,d3,t15,1.0
6553,i2,j4,d1,t18,1.0
11947,i3,j3,d4,t21,1.0
17266,i4,j3,d1,t23,1.0
23707,i5,j5,d3,t2,1.0
28734,i6,j4,d2,t35,1.0
34523,i7,j5,d1,t36,1.0
38152,i8,j1,d2,t37,1.0
47412,i9,j10,d1,t1,1.0
53258,i10,j10,d3,t13,1.0


In [57]:
df_dict = {
    (row.n, row.nn, row.d): row.t
    for row in last_connections.itertuples(index=False)
}

In [59]:
ixlm_values

{('pw1', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw2', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw3', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw4', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw5', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw6', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw7', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw8', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw9', 'j1', 'pf1', 'd1', 't1'): None,
 ('pw10', 'j1', 'pf1', 'd1', 't1'): None}

In [58]:
df_dict

{('i1', 'j1', 'd3'): 't15',
 ('i2', 'j4', 'd1'): 't18',
 ('i3', 'j3', 'd4'): 't21',
 ('i4', 'j3', 'd1'): 't23',
 ('i5', 'j5', 'd3'): 't2',
 ('i6', 'j4', 'd2'): 't35',
 ('i7', 'j5', 'd1'): 't36',
 ('i8', 'j1', 'd2'): 't37',
 ('i9', 'j10', 'd1'): 't1',
 ('i10', 'j10', 'd3'): 't13',
 ('i11', 'j10', 'd1'): 't16',
 ('i12', 'j10', 'd1'): 't19',
 ('j1', 'pf2', 'd2'): 't15',
 ('j3', 'pf2', 'd3'): 't21',
 ('j4', 'pf2', 'd3'): 't18',
 ('j5', 'pf2', 'd3'): 't2',
 ('j10', 'pf2', 'd2'): 't1'}